<a href="https://colab.research.google.com/github/entmike/disco-diffusion-1/blob/main/View%20Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖼️ Simplified Disco Diffusion (Batch Viewer)

[![Discord](https://badgen.net/badge/icon/discord?icon=discord&label)](https://discord.gg/FqsE83fT)
[![Maintenance](https://img.shields.io/badge/Maintained%3F-yes-green.svg)](https://github.com/entmike/disco-diffusion-1)
![Terminal](https://badgen.net/badge/icon/terminal?icon=terminal&label)
[![Issues](https://img.shields.io/github/issues/entmike/disco-diffusion-1)](https://github.com/entmike/disco-diffusion-1/issues)
![Stars](https://img.shields.io/github/stars/entmike/disco-diffusion-1)
![Commits](https://img.shields.io/github/commit-activity/w/entmike/disco-diffusion-1)

**Expand this section for details.**

Special thanks to Zippy#1111 on Discord (https://gist.github.com/aredden) for image grid ninja work!

In [3]:
import subprocess
use_google_drive = True #@param {type:"boolean"}
dbpath = "/content/gdrive/MyDrive/disco-diffusion-1/disco.db" #@param {type:"string"}
if use_google_drive == True:
  import os
  from google.colab import drive
  if os.path.isdir('/content/gdrive') == False:
    print(f'📁 Mounting Google Drive.  Please accept any confirmation screens.')
    drive.mount('/content/gdrive/')
  else:
    print(f'📁 Google Drive already mounted.')
  content_root = '/content/gdrive/MyDrive'

print(f'📦 Installing sqlite3...')
subprocess.run(f'pip install --upgrade pyyaml --quiet'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")

📁 Mounting Google Drive.  Please accept any confirmation screens.
Mounted at /content/gdrive/
📦 Installing sqlite3...


''

In [4]:
import sqlite3
import io
import numpy as np
from IPython.display import Image, display
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image as PILImg
from typing import List
import cv2

dbcon = sqlite3.connect(dbpath)
cursor = dbcon.cursor()

def fetchImages(sql = None):
    cursor.execute(sql)
    records = cursor.fetchall()
    images = []
    for row in records:
        blob_data = row[0]
        # nparr = np.fromstring(blob_data, np.uint8)
        images.append([np.array(PILImg.open(io.BytesIO(blob_data))), row[1]])
    return images

def image_grid(images: List[np.ndarray], ncols: int, scale=0.5):
    """
    Creates a grid of images with 'ncols' number of image columns, images must all be the same shape.
    """
    assert len(images) >= ncols, "Must have more images than columns"
    nrows = math.ceil(len(images) / ncols)
    images_size = images[0][0].shape[:2]
    grid_img = np.zeros((nrows * images_size[0], ncols * images_size[1], 3), dtype=np.uint8)
    for i, img in enumerate(images):
        # image, name = img
        img = cv2.putText(
            img=img[0].astype(np.uint8),
            text=img[1].split("/")[-1],
            org=(40, 50),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=2,
            color=(255,255,255),
            thickness=4,
            lineType=cv2.LINE_8,
        )
        r = int(i / ncols)
        c = i % ncols
        grid_img[r * images_size[0]:(r + 1) * images_size[0], c * images_size[1]:(c + 1) * images_size[1]] = img
    if scale != 1:
        grid_img = cv2.resize(grid_img, (0,0), fx=scale, fy=scale,interpolation=cv2.INTER_LANCZOS4)
    return grid_img

def displayGrid(image):
    fig = plt.figure(dpi=300)
    plot = fig.add_subplot(111)
    plot.axis('off')
    plot.imshow(image)
    plot.set_xlim(0,image.shape[1])
    plot.set_ylim(image.shape[0],0)
    plot.set_alpha(1)
    plot.set_clip_on(True)
    plot.figure.canvas.draw()

# Display Images in a Grid

In [ ]:
session_id = '5577d7ed-07c3-4dac-8e3d-b08c4dea8b7d'
images = fetchImages(sql = f"SELECT images.image, images.image_path FROM jobs INNER JOIN images ON images.job_uuid = jobs.uuid WHERE jobs.session_uuid='{session_id}'")
print(f'{len(images)} images found.')
grid = image_grid(images, 4)
displayGrid(grid)